In [6]:
import glob

# function to extract the paths for files froma path
def load_paths(data_path):
    files = []
    files.append(glob.glob(data_path, 
                recursive = True))
    return files

In [3]:
import os

path = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/text/Participant_6/Processed_data/Video/Subject_06_05.mp4"
print("file exists?", os.path.exists(path))

file exists? False


In [9]:
data_path = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/notext/"
files = load_paths(data_path + "*/Processed_data/Video/*.mp4")

In [10]:
files

[['C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/notext\\Participant_12_no_labels\\Processed_data\\Video\\Subject_12_01.mp4',
  'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/notext\\Participant_12_no_labels\\Processed_data\\Video\\Subject_12_02.mp4',
  'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/notext\\Participant_12_no_labels\\Processed_data\\Video\\Subject_12_03.mp4',
  'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/notext\\Participant_12_no_labels\\Processed_data\\Video\\Subject_12_04.mp4',
  'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/notext\\Participant_12_no_labels\\Processed_data\\Video\\Subject_12_05.mp4',
  'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/notext\\Participant_12_no_labels\\Processed_data\\Video\\Subject_12_06.mp4',
  'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/notext\\Participant_12_no_labels\\Processed_data\\Video\\Subject_12_0

In [3]:
import cv2
import pytesseract
import random
import string

def detect_text_change(video_path, save_path):
    cap = cv2.VideoCapture(video_path)
    fg_bg = cv2.createBackgroundSubtractorMOG2()
    text_on = False
    count = 0
    current_text = ""
    out = None
    videos = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        fg_mask = fg_bg.apply(frame)
        white_pixels = cv2.countNonZero(fg_mask[200:, -200:])
        
        # if white_pixels > 0 and not text_on:
        #     text_on = True
        #     text = ''
        #     text += " " + pytesseract.image_to_string(frame[400:, 900:])
        #     current_text = text.strip()
        #     out = cv2.VideoWriter(f"{save_path}/{current_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))
        #     print(f"Text {current_text} on")
        #     start_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        
        # if white_pixels == 0 and text_on:
        #     text_on = False
        #     out.release()
        #     print(f"Text {current_text} off")
        #     end_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
        #     videos.append((start_frame, end_frame))

        if white_pixels > 0 and not text_on:
            text_on = True
            text = ''
            text += " " + pytesseract.image_to_string(frame[400:, 900:])
            current_text = text.strip()
            # first word
            if out is None:
                # if the word is not recognised then name the video with a random letter
                # if current_text == '':
                #     current_text=''.join(random.choice(string.ascii_lowercase))
                    # write out
                if current_text == '':
                    #     current_text=''.join(random.choice(string.ascii_lowercase))
                    add_text = ''.join(random.choice(string.ascii_lowercase))
                    out = cv2.VideoWriter(f"{save_path}/{current_text + add_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))

                else: out = cv2.VideoWriter(f"{save_path}/{current_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))
                print('out is None')
            # elif current_text == prev_text:

            else:
                if current_text != prev_text:
                    out.release()
                    if current_text == '':
                    #     current_text=''.join(random.choice(string.ascii_lowercase))
                        add_text = ''.join(random.choice(string.ascii_lowercase))
                        out = cv2.VideoWriter(f"{save_path}/{current_text + add_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))

                    else: out = cv2.VideoWriter(f"{save_path}/{current_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))
                    print('current_text != prev')
                print('else')
            prev_text = current_text
            print(f"Text {current_text} on")
            start_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        
        if white_pixels == 0 and text_on:
            text_on = False
            out.release()
            print(f"Text {current_text} off")
            # end_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
            end_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
            if current_text == '':
                videos.append((current_text + add_text, start_frame, end_frame))
                # add_text = ''
            else: videos.append((current_text, start_frame, end_frame))
        
        if text_on:
            out.write(frame)
            
        cv2.imshow('Frame', frame)
        cv2.imshow('FG Mask', fg_mask)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return videos



# def detect_text_change(video_path, save_path):
#     cap = cv2.VideoCapture(video_path)
#     fg_bg = cv2.createBackgroundSubtractorMOG2()
#     text_on = False
#     current_text = ""
#     out = None
#     videos = []

#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break

#         fg_mask = fg_bg.apply(frame)
#         white_pixels = cv2.countNonZero(fg_mask[200:, -200:])

#         if white_pixels > 0 and not text_on:
#             text_on = True
#             text = ''
#             text += " " + pytesseract.image_to_string(frame[400:, 900:])
#             current_text = text.strip()
#             if out is None:
#                 out = cv2.VideoWriter(f"{save_path}/{current_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))
#                 start_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
#             else:
#                 if current_text != prev_text:
#                     out.release()
#                     end_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
#                     videos.append((start_frame, end_frame))
#                     start_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
#                     out = cv2.VideoWriter(f"{save_path}/{current_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))
#             prev_text = current_text
#             print(f"Text {current_text} on")

#         if white_pixels == 0 and text_on:
#             text_on = False
#             out.release()
#             end_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
#             videos.append((start_frame, end_frame))
#             print(f"Text {current_text} off")

#         if text_on:
#             out.write(frame)

#         cv2.imshow('Frame', frame)
#         cv2.imshow('FG Mask', fg_mask)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()
#     return videos



# def detect_text_change(video_path, save_path):
#     cap = cv2.VideoCapture(video_path)
#     fg_bg = cv2.createBackgroundSubtractorMOG2()
#     text_on = False
#     count = 0
#     current_text = ""
#     out = None
#     videos = []

#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break

#         fg_mask = fg_bg.apply(frame)
#         white_pixels = cv2.countNonZero(fg_mask[200:, -200:])
#         cv2.imshow(fg_mask[200:, -200:])    

#         if white_pixels > 0 and not text_on:
#             text_on = True
#             text = ''
#             text += " " + pytesseract.image_to_string(frame[400:, 900:])
#             current_text = text.strip()
#             if out is None:
#                 out = cv2.VideoWriter(f"{save_path}/{current_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))
#             else:
#                 if current_text != prev_text:
#                     out.release()
#                     end_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
#                     videos.append((start_frame, end_frame))
#                     start_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
#                     out = cv2.VideoWriter(f"{save_path}/{current_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))
#             prev_text = current_text
#             print(f"Text {current_text} on")
#             start_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        
#         if white_pixels == 0 and text_on:
#             text_on = False
#             out.release()
#             end_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
#             videos.append((start_frame, end_frame))
#             print(f"Text {current_text} off")
        
#         if text_on:
#             out.write(frame)
            
#         cv2.imshow('Frame', frame)
#         cv2.imshow('FG Mask', fg_mask)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()
#     return videos

In [60]:
import cv2
import pytesseract

def detect_text_change(video_path, save_path):
    cap = cv2.VideoCapture(video_path)
    fg_bg = cv2.createBackgroundSubtractorMOG2()
    text_on = False
    count = 0
    current_text = ""
    prev_text = ""
    out = None
    videos = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        fg_mask = fg_bg.apply(frame)
        white_pixels = cv2.countNonZero(fg_mask[200:, -200:])

        if white_pixels > 0 and not text_on:
            text_on = True
            text = ''
            text += " " + pytesseract.image_to_string(frame[400:, 900:])
            current_text = text.strip()
            # if out is None:
            #     out = cv2.VideoWriter(f"{save_path}/{current_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))
            # else:
            if current_text != prev_text:
                    out.release()
                    out = cv2.VideoWriter(f"{save_path}/{current_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))
            # elif out is None:
            #     out = cv2.VideoWriter(f"{save_path}/{current_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))
            prev_text = current_text
            if out is None:
                out = cv2.VideoWriter(f"{save_path}/{current_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))
            print(f"Text {current_text} on")
            start_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

        if white_pixels == 0 and text_on:
            text_on = False
            out.release()
            print(f"Text {current_text} off")
            end_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
            videos.append((current_text, start_frame, end_frame))
            current_text = ""
            prev_text = ""

        if text_on:
            out.write(frame)

        cv2.imshow('Frame', frame)
        cv2.imshow('FG Mask', fg_mask)
        cv2.imshow('fg cut', fg_mask[200:, -200:])
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return videos

In [4]:
path = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/text/Participant_10/Processed_data/Video/Subject_10_05.mp4"
save = "C:/Users/men22/Documents/Videos3/"
stamps = detect_text_change(path, save)
stamps

out is None
Text  on
Text  off
else
Text  on
Text  off
current_text != prev
else
Text Pen on
Text Pen off
else
Text Pen on
Text Pen off
current_text != prev
else
Text Bigail on
Text Bigail off
else
Text Bigail on
Text Bigail off
current_text != prev
else
Text  on
Text  off
current_text != prev
else
Text Desg on
Text Desg off
else
Text Desg on
Text Desg off
current_text != prev
else
Text Cath on
Text Cath off
current_text != prev
else
Text Galw on
Text Galw off
else
Text Galw on
Text Galw off
current_text != prev
else
Text Ffair on
Text Ffair off
else
Text Ffair on
Text Ffair off
current_text != prev
else
Text Faint on
Text Faint off
else
Text Faint on
Text Faint off
current_text != prev
else
Text Theatr on
Text Theatr off
else
Text Theatr on
Text Theatr off
current_text != prev
else
Text DY s fer) on
Text DY s fer) off
current_text != prev
else
Text  on
Text  off
else
Text  on
Text  off
else
Text  on
Text  off
current_text != prev
else
Text Llaw on
Text Llaw off
else
Text Llaw on
Text 

[('r', 1, 1),
 ('r', 31, 37),
 ('Pen', 106, 176),
 ('Pen', 200, 222),
 ('Bigail', 277, 351),
 ('Bigail', 372, 394),
 ('l', 468, 585),
 ('Desg', 639, 718),
 ('Desg', 733, 755),
 ('Cath', 821, 938),
 ('Galw', 987, 1061),
 ('Galw', 1082, 1104),
 ('Ffair', 1185, 1262),
 ('Ffair', 1279, 1301),
 ('Faint', 1365, 1442),
 ('Faint', 1460, 1482),
 ('Theatr', 1549, 1624),
 ('Theatr', 1644, 1666),
 ('DY s fer)', 1725, 1842),
 ('n', 1897, 1974),
 ('n', 1981, 1983),
 ('n', 1992, 2014),
 ('Llaw', 2073, 2154),
 ('Llaw', 2168, 2190),
 ('Siop', 2266, 2383),
 ('Chi', 2438, 2514),
 ('Chi', 2521, 2521),
 ('Chi', 2533, 2555),
 ('Het', 2629, 2719),
 ('Het', 2724, 2746),
 ('Cwtsh', 2802, 2919),
 ('Jam', 2976, 3052),
 ('Jam', 3071, 3093),
 ('Rhaw', 3170, 3246),
 ('Rhaw', 3265, 3287),
 ('sXeloleya', 3350, 3424),
 ('sXeloleya', 3445, 3467),
 ('Lamp', 3527, 3620),
 ('Lamp', 3622, 3644),
 ('Mawr', 3698, 3775),
 ('Mawr', 3793, 3815),
 ('Neidr', 3888, 3963),
 ('Neidr', 3983, 4005),
 ('Cyngor', 4070, 4162),
 ('Cyngor'

In [5]:
stamps

[('r', 1, 1),
 ('r', 31, 37),
 ('Pen', 106, 176),
 ('Pen', 200, 222),
 ('Bigail', 277, 351),
 ('Bigail', 372, 394),
 ('l', 468, 585),
 ('Desg', 639, 718),
 ('Desg', 733, 755),
 ('Cath', 821, 938),
 ('Galw', 987, 1061),
 ('Galw', 1082, 1104),
 ('Ffair', 1185, 1262),
 ('Ffair', 1279, 1301),
 ('Faint', 1365, 1442),
 ('Faint', 1460, 1482),
 ('Theatr', 1549, 1624),
 ('Theatr', 1644, 1666),
 ('DY s fer)', 1725, 1842),
 ('n', 1897, 1974),
 ('n', 1981, 1983),
 ('n', 1992, 2014),
 ('Llaw', 2073, 2154),
 ('Llaw', 2168, 2190),
 ('Siop', 2266, 2383),
 ('Chi', 2438, 2514),
 ('Chi', 2521, 2521),
 ('Chi', 2533, 2555),
 ('Het', 2629, 2719),
 ('Het', 2724, 2746),
 ('Cwtsh', 2802, 2919),
 ('Jam', 2976, 3052),
 ('Jam', 3071, 3093),
 ('Rhaw', 3170, 3246),
 ('Rhaw', 3265, 3287),
 ('sXeloleya', 3350, 3424),
 ('sXeloleya', 3445, 3467),
 ('Lamp', 3527, 3620),
 ('Lamp', 3622, 3644),
 ('Mawr', 3698, 3775),
 ('Mawr', 3793, 3815),
 ('Neidr', 3888, 3963),
 ('Neidr', 3983, 4005),
 ('Cyngor', 4070, 4162),
 ('Cyngor'

In [15]:
def extract_segments2(video_path, segments, output_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    stamps1 = segments

    # define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None

    # for i, segment in enumerate(segments):
    #     start_frame, end_frame = segment
    #     if end_frame-start_frame<=3:break

    #     # check if difference between start and end frame is less than 25
    #     if out is not None and start_frame - last_end_frame <= 30:
    #         # join current segment to previous segment
    #         out.release()
    #         out = None
    #         segments[i] = (last_start_frame, end_frame)
    #         continue
    #     if out is not None and i+1 < len(segments) and segments[i+1][0] - segments[i+1][1] <= 30: 
    #         # join current segment to previous segment
    #         out.release()
    #         out = None
    #         segments[i] = (last_start_frame, segments[i+1][1])
    #         segments.pop(i+1)
    #         continue


    for i, segment in enumerate(segments):
        current_text, start_frame, end_frame = segment
        # if end_frame-start_frame<=3:break

        # check if difference between start and end frame is less than 25
        if out is not None and prev_text == current_text:
            # join current segment to previous segment
            out.release()
            out = None
            segments[i] = (current_text, last_start_frame, end_frame)
            if i+1 < len(segments) and segments[i+1][0] == current_text: 
                segments[i] = (current_text, last_start_frame, segments[i+1][2])
                segments.pop(i+1)
            continue


        # extract video segment
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        out = cv2.VideoWriter(f"{output_path}/segment_{current_text}.mp4", fourcc, 30, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
        for frame_idx in range(start_frame, end_frame+1):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

        last_start_frame, last_end_frame, prev_text = start_frame, end_frame, current_text
        out.release()

    cap.release()
    return stamps1

In [30]:
def extract_segments3(video_path, segments, output_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    # define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = []

    for i, segment in enumerate(segments):
        start_frame, end_frame = segment

        # check if difference between start and end frame is less than 30
        if out and start_frame - last_end_frame <= 30:
            # join current segment to previous segment
            out[-1].release()
            out.pop()
            segments[i] = (last_start_frame, end_frame)
            continue

        # check if difference between current segment and next segment is less than 30
        if i + 1 < len(segments):
            next_start_frame, next_end_frame = segments[i+1]
            if next_start_frame - end_frame <= 30:
                # join current segment and next segment
                out[-1].release()
                out.pop()
                segments[i] = (start_frame, next_end_frame)
                continue

        # check if difference between current segment and the segment after next is less than 30
        if i + 2 < len(segments):
            next_next_start_frame, next_next_end_frame = segments[i+2]
            if next_next_start_frame - end_frame <= 30:
                # join current segment, next segment, and segment after next
                out[-1].release()
                out.pop()
                segments[i] = (start_frame, next_next_end_frame)
                segments.pop(i+1)
                continue

        # extract video segment
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        out.append(cv2.VideoWriter(f"{output_path}/segment_{i}.mp4", fourcc, 30, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))))
        for frame_idx in range(start_frame, end_frame+1):
            ret, frame = cap.read()
            if not ret:
                break
            out[-1].write(frame)

        last_start_frame, last_end_frame = start_frame, end_frame
        out[-1].release()

    cap.release()

In [35]:
def extract_segments3(video_path, segments, output_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    # define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None

    for i, segment in enumerate(segments):
        start_frame, end_frame = segment

        # check if difference between start and end frame is less than 30 for two segments
        if out is not None and i+1 < len(segments) and start_frame - last_end_frame <= 30 and segments[i+1][1] - start_frame <= 30:
            # join current segment and next segment
            out.release()
            out = None
            segments[i] = (last_start_frame, segments[i+1][1])
            segments.pop(i+1)
            continue

        # extract video segment
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        out = cv2.VideoWriter(f"{output_path}/segment_{i}.mp4", fourcc, 30, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
        for frame_idx in range(start_frame, end_frame+1):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

        last_start_frame, last_end_frame = start_frame, end_frame
        out.release()

    cap.release()

In [17]:
path = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/text/Participant_9/Processed_data/Video/Subject_09_05.mp4"
save = "C:/Users/men22/Documents/Videos3/"
segemnts = extract_segments2(path, stamps, save)
segemnts

[('r', 1, 1),
 ('r', 1, 37),
 ('Pen', 106, 176),
 ('Pen', 106, 222),
 ('Bigail', 277, 351),
 ('Bigail', 277, 394),
 ('l', 468, 585),
 ('Desg', 639, 718),
 ('Desg', 639, 755),
 ('Cath', 821, 938),
 ('Galw', 987, 1061),
 ('Galw', 987, 1104),
 ('Ffair', 1185, 1262),
 ('Ffair', 1185, 1301),
 ('Faint', 1365, 1442),
 ('Faint', 1365, 1482),
 ('Theatr', 1549, 1624),
 ('Theatr', 1549, 1666),
 ('DY s fer)', 1725, 1842),
 ('n', 1897, 1974),
 ('n', 1897, 2014),
 ('Llaw', 2073, 2154),
 ('Llaw', 2073, 2190),
 ('Siop', 2266, 2383),
 ('Chi', 2438, 2514),
 ('Chi', 2438, 2555),
 ('Het', 2629, 2719),
 ('Het', 2629, 2746),
 ('Cwtsh', 2802, 2919),
 ('Jam', 2976, 3052),
 ('Jam', 2976, 3093),
 ('Rhaw', 3170, 3246),
 ('Rhaw', 3170, 3287),
 ('sXeloleya', 3350, 3424),
 ('sXeloleya', 3350, 3467),
 ('Lamp', 3527, 3620),
 ('Lamp', 3527, 3644),
 ('Mawr', 3698, 3775),
 ('Mawr', 3698, 3815),
 ('Neidr', 3888, 3963),
 ('Neidr', 3888, 4005),
 ('Cyngor', 4070, 4162),
 ('Cyngor', 4070, 4187),
 ('laith', 4252, 4344),
 ('la